#### 1. Import pandas library

In [50]:
import pandas as pd

#### 2. Import BigQuery library


In [1]:
from google.cloud import bigquery


#### 3. Create a connection to BigQuery

In [6]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/Myrto.Setzi/data-labs/module-1/lab-data_cleaning/your-code/Ironhack Data Analytics.json"

client = bigquery.Client()

#### 4. Import data from the users table from the stackoverflow public database for United Kingdom only

In [11]:
query_users='''
SELECT 
    * 
FROM 
    `bigquery-public-data.stackoverflow.users`
WHERE
    location='United Kingdom'
'''
query_job = client.query(query=query_users)

In [12]:
users=query_job.to_dataframe()

#### 5. Rename id column to user_id

In [15]:
users = users.rename(columns={'id': 'user_id'})

In [16]:
users.head()

,user_id,display_name,about_me,age,creation_date,last_access_date,location,reputation,up_votes,down_votes,views,profile_image_url,website_url
0,32777,dmcnally,,,2008-10-30 14:40:32.323000+00:00,2019-05-22 06:53:30.717000+00:00,United Kingdom,731,16,6,38,,http://artofsource.com
1,77741,Jimit,,,2009-03-13 14:16:47.703000+00:00,2019-05-29 09:49:12.330000+00:00,United Kingdom,435,28,1,110,,
2,174979,Justin Emery,<p>I mostly work with web technologies. Lots o...,,2009-09-17 14:05:34.943000+00:00,2019-05-29 12:38:46.327000+00:00,United Kingdom,916,186,19,91,https://www.gravatar.com/avatar/f6354e6366838e...,http://www.justinemery.co.uk
3,1502608,David Tew,,,2012-07-04 22:59:03.567000+00:00,2019-05-19 19:55:32.693000+00:00,United Kingdom,1264,44,0,90,,http://www.drpbanerji.com
4,2550349,ArthurChamz,<p>I'm a software engineer. That's all for now...,,2013-07-04 11:52:06.953000+00:00,2019-05-31 10:35:48.813000+00:00,United Kingdom,1674,398,8,170,https://i.stack.imgur.com/M3Tx5.jpg?s=128&g=1,http://mx.linkedin.com/in/arturocastroperpuli/


#### 6. Import from the stackoverflow_posts table all items which have up to 20 view counts and only select 'id', 'owner_user_id', 'score', 'view_count', 'comment_count', 'favorite_count'

In [22]:
query_posts='''
SELECT 
    id, 
    owner_user_id,
    score,
    view_count,
    comment_count,
    favorite_count
FROM 
    `bigquery-public-data.stackoverflow.stackoverflow_posts`
WHERE
    view_count<=20
'''
query_job = client.query(query=query_posts)

In [23]:
posts=query_job.to_dataframe()

#### 7. Rename id column to post_id and owner_user_id to user_id

In [24]:
posts = posts.rename(columns={'id': 'post_id','owner_user_id': 'user_id'})

#### 8. Define new dataframes for users and posts with the following selected columns:
    **users columns**: user_id, reputation,views,up_votes,down_votes
    **posts columns**: post_id, score,user_id,view_count,comment_count

In [26]:
users = users[['user_id','reputation','views','up_votes','down_votes']]
posts = posts[['post_id','user_id','score','view_count','comment_count']]

users.head()

,user_id,reputation,views,up_votes,down_votes
0,32777,731,38,16,6
1,77741,435,110,28,1
2,174979,916,91,186,19
3,1502608,1264,90,44,0
4,2550349,1674,170,398,8


In [27]:
posts.head()

,post_id,user_id,score,view_count,comment_count
0,24292513,NaN,0,19,7
1,35155703,NaN,0,16,0
2,35029414,3479566.0,0,16,1
3,33501255,NaN,1,14,0
4,31512834,NaN,0,12,2


#### 8. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [29]:
df_merged = users.merge(posts, on='user_id')

In [43]:
df_merged.head()

,user_id,reputation,views,up_votes,down_votes,post_id,score,view_count,comment_count
0,2550349,1674,170,398,8,36705627,2,20,1
1,2550349,1674,170,398,8,36799577,0,6,0
2,401743,759,76,8,2,35922834,0,13,0
3,1145123,454,59,54,0,24964496,0,17,0
4,1046182,326,81,10,0,36516756,0,20,1


#### 9. How many missing values do you have in your merged dataframe? 

In [33]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1088 entries, 0 to 1087
Data columns (total 9 columns):
user_id          1088 non-null object
reputation       1088 non-null int64
views            1088 non-null int64
up_votes         1088 non-null int64
down_votes       1088 non-null int64
post_id          1088 non-null int64
score            1088 non-null int64
view_count       1088 non-null int64
comment_count    1088 non-null int64
dtypes: int64(8), object(1)
memory usage: 85.0+ KB


In [47]:
df_merged.isnull().sum()

user_id          0
reputation       0
views            0
up_votes         0
down_votes       0
post_id          0
score            0
view_count       0
comment_count    0
dtype: int64

There are no missing values

#### Bonus: Identify extreme values in your merged dataframe, create a dataframe called outliers with the same columns as our data set and calculate the bounds. The values of the outliers dataframe will be the values of the merged_df that fall outside that bounds. You will need to save your outliers dataframe to a csv file on your-code folder. Hint: post_id cannot have outliers!

In [55]:
stats = df_merged.loc[:, df_merged.columns != 'post_id'].describe().T
stats['IQR'] = stats['75%'] - stats['25%']
stats

,count,mean,std,min,25%,50%,75%,max,IQR
reputation,1088.0,4815.090074,15438.390331,1.0,139.0,774.0,3128.25,263224.0,2989.25
views,1088.0,594.238051,1568.560266,0.0,37.0,135.0,462.25,26310.0,425.25
up_votes,1088.0,384.477941,730.548850,0.0,16.0,89.0,388.25,7664.0,372.25
down_votes,1088.0,47.390625,167.131398,0.0,0.0,3.0,20.00,2440.0,20.00
score,1088.0,0.125000,0.588919,-7.0,0.0,0.0,0.00,4.0,0.00
view_count,1088.0,15.566176,3.699031,3.0,13.0,16.0,19.00,20.0,6.00
comment_count,1088.0,0.927390,1.641589,0.0,0.0,0.0,1.00,11.0,1.00


In [56]:
outliers = pd.DataFrame(columns=df_merged.columns)

In [57]:
for col in stats.index:
    iqr = stats.at[col,'IQR']
    cutoff = iqr * 1.5
    lower = stats.at[col,'25%'] - cutoff
    upper = stats.at[col,'75%'] + cutoff
    results = df_merged[(df_merged[col] < lower) | 
                   (df_merged[col] > upper)].copy()
    results['Outlier'] = col
    outliers = outliers.append(results)

In [63]:
outliers.to_csv('outliers.csv')